### PREDICTING THE BEST LOCATION FOR A COFFEE STALL IN MANHATTAN

### INTRODUCTION

The problem or question to be asked is "what is the best neighbourhood of Manhattan to open a coffee stall?"

Good locations are those with a high foot traffic but with the lowest competition.

A good area would have a high foot traffic, but a low number of coffee shops.

Audience of the results will be entrepreneurs looking to set up a coffee stall in Manhattan.

They care about this question/problem because Manhattan is already a competitve place for coffee shops, but opening shop in a neighbourhood with high foot traffic and low competition will be the area most likely to return a profit.

In [1]:
!conda install lxml --yes

import pandas as pd

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2020.6.2~ --> pkgs/main::ca-certificates-2020.7.22-0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2020.6.20-py36h9~ --> pkgs/main::certifi-2020.6.20-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_1 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ con

### DATA

Three datasets willl be used:

1) List of Manhattan neighbourhoods from wikipedia

https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods#:~:text=The%20following%20approximate%20definitions%20are,34th%20Street%20and%2059th%20Street.

bringing in the following string data

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods', header=0)
pd.set_option('display.max_rows', 200)
df_upper = data[0]
df_midtown = data[1]
df_between = data[2]
df_downtown = data[3]
frames = [df_upper, df_midtown, df_between, df_downtown]
df_manhattan = pd.concat(frames)
df_manhattan.reset_index(drop = True, inplace=True)

Datatypes of df_manhattan

In [3]:
df_manhattan.dtypes

Name of the neighborhood                  object
Limits south to north and east to west    object
dtype: object

Some Neighbourhood name entries contain brackets, square brackets and daggers, which will interfere with the process of assigning a neighbourhood name to the foot traffic samples points.

In [4]:
print(df_manhattan["Name of the neighborhood"].loc[46])
print(df_manhattan["Name of the neighborhood"].loc[62])
print(df_manhattan["Name of the neighborhood"].loc[77])

Flower District[3][4][5]
Little Germany (historic)
Cooperative Village†


Removing bracketed substrings in names, square bracketed footnote references, daggers in names and finally the 

In [5]:
df_manhattan["Name of the neighborhood"] = df_manhattan["Name of the neighborhood"].str.replace(r"\(.*\)","")
df_manhattan["Name of the neighborhood"] = df_manhattan["Name of the neighborhood"].str.replace(r"\[.*\]","")
df_manhattan["Name of the neighborhood"] = df_manhattan["Name of the neighborhood"].str.replace(u"\N{DAGGER}","")
df_manhattan["Name of the neighborhood"] = df_manhattan["Name of the neighborhood"].str.strip()

Giving example of df_manhattan database

In [6]:
print(df_manhattan["Name of the neighborhood"].loc[46])
print(df_manhattan["Name of the neighborhood"].loc[62])
print(df_manhattan["Name of the neighborhood"].loc[77])

Flower District
Little Germany
Cooperative Village


Description of Manhattan Neighbourhoods

In [8]:
df_manhattan["Name of the neighborhood"].describe()

count               85
unique              83
top       Hudson Yards
freq                 2
Name: Name of the neighborhood, dtype: object

2) Pedestrian Traffic data from NYC Open Data

https://data.cityofnewyork.us/api/views/cqsj-cfgu/rows.csv?accessType=DOWNLOAD&bom=true&format=true

The dataset has the following relevant columns:

Borough (e.g. Manhattan) - String
the_geom (lat, lng coordinate) - String
Time series columns (e.g. 1576 for May 2019 at 161 street in the bronx) - Integer

For each year from 2007 to 2019 measurements of pedestrian data is taken bi annually in May and September at morning, midday and afternoon. 

Data for 2019 only will be averaged for each neighbourhood. 

The GeoPy Library will be used to assign an area name to each lat and lng value in the Pedestrian Traffic Data using the Reverse function. If a matching neighbourhood address cannot be assigned, one will be assigned manually.

In [9]:
df_ped = pd.read_csv('https://data.cityofnewyork.us/api/views/cqsj-cfgu/rows.csv?accessType=DOWNLOAD&bom=true&format=true')
df_ped = df_ped[(df_ped.Borough=='Manhattan')]

Examples of pedestrian Traffic data 

In [10]:
df_ped.head()

,Borough,the_geom,OBJECTID,Loc,Street_Nam,From_Stree,To_Street,Index,May07_AM,May07_PM,...,Sept17_PM,Sept17_MD,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,May19_AM,May19_PM,May19_MD
34,Manhattan,POINT (-74.01155687409947 40.70463665187371),35,35,Broad Street,Beaver Street,South William Street,Y,3469,3992,...,"8,303","2,036","4,374","6,603","1,756","6,471","2,010","4,100","7,302","1,669"
35,Manhattan,POINT (-74.01286204592034 40.70634164448266),36,36,Broadway,Morris Street,Exchange Place,Y,3660,8390,...,"12,650","8,126","5,221","19,725","6,818","10,726","9,615","5,049","11,765","7,029"
36,Manhattan,POINT (-73.98219706247882 40.77181340301184),37,37,Broadway,West 63rd Street,West 64th Street,Y,1611,6764,...,"9,305","4,663","2,059","6,194","6,037","7,773","5,259","1,696","6,864","4,907"
37,Manhattan,POINT (-74.01009312926121 40.715904559004194),38,38,Chambers Street,West Broadway,Greenwich Street,Y,7081,8512,...,"9,937","3,302","8,323","8,960","3,630","10,456","3,493","3,075","6,598","2,934"
38,Manhattan,POINT (-73.97713579908014 40.7796808276313),39,39,Columbus Avenue,West 75th Street,West 76th Street,N,1071,3037,...,"3,626","2,977","1,524","3,905","3,780","3,794","2,451",-,-,-


In [11]:
df_ped.dtypes

Borough       object
the_geom      object
OBJECTID       int64
Loc            int64
Street_Nam    object
From_Stree    object
To_Street     object
Index         object
May07_AM      object
May07_PM      object
May07_MD      object
Sept07_AM     object
Sept07_PM     object
Sept07_MD     object
May08_AM      object
May08_PM      object
May08_MD      object
Sept08_AM     object
Sept08_PM     object
Sept08_MD     object
May09_AM       int64
May09_PM       int64
May09_MD       int64
Sept09_AM     object
Sept09_PM     object
Sept09_MD     object
May10_AM       int64
May10_PM       int64
May10_MD       int64
Sept10_AM      int64
Sept10_PM      int64
Sept10_MD      int64
May11_AM       int64
May11_PM       int64
May11_MD       int64
Sept11_AM      int64
Sept11_PM      int64
Sept11_MD      int64
May12_AM       int64
May12_PM       int64
May12_MD       int64
Sept12_AM     object
Sept12_PM     object
Sept12_MD     object
May13_AM      object
May13_PM      object
May13_MD      object
Sept13_AM    

Description of pedestrian traffic data

In [12]:
df_ped.describe()

,OBJECTID,Loc,May09_AM,May09_PM,May09_MD,May10_AM,May10_PM,May10_MD,Sept10_AM,Sept10_PM,...,May11_MD,Sept11_AM,Sept11_PM,Sept11_MD,May12_AM,May12_PM,May12_MD,Sept15_AM,Sept15_PM,Sept15_MD
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,...,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,52.500000,52.500000,3709.666667,9949.138889,4485.750000,4096.194444,11097.277778,4615.277778,4033.916667,10709.888889,...,4989.111111,4318.944444,10446.722222,5140.527778,4120.861111,7514.805556,5268.638889,4492.611111,11652.833333,4686.000000
std,10.535654,10.535654,2664.188829,6881.739540,3273.855022,2911.466757,6968.435994,2991.685064,2498.866673,6727.910071,...,3196.104690,3210.427448,5806.368910,3488.050151,2830.395946,4576.350004,3474.177437,2505.843482,5938.561180,3008.181302
min,35.000000,35.000000,27.000000,192.000000,64.000000,280.000000,453.000000,201.000000,273.000000,313.000000,...,487.000000,233.000000,141.000000,1515.000000,255.000000,118.000000,1175.000000,281.000000,694.000000,733.000000
25%,43.750000,43.750000,1543.750000,5762.250000,2240.500000,1986.500000,6151.750000,2591.750000,2378.500000,6141.500000,...,2700.500000,1993.250000,5882.000000,2435.750000,2101.000000,4280.250000,2704.500000,2349.000000,7205.250000,2191.000000
50%,52.500000,52.500000,3424.000000,8518.000000,3557.500000,3907.500000,9961.000000,3831.000000,3693.500000,9001.000000,...,4216.500000,3759.000000,10063.500000,4401.000000,3753.000000,6798.000000,4333.500000,4088.000000,11341.500000,3625.000000
75%,61.250000,61.250000,4778.750000,11090.750000,5465.000000,5312.000000,13007.250000,5981.750000,4782.750000,13997.750000,...,6006.500000,5812.500000,12380.750000,6709.000000,5608.500000,8990.000000,6343.500000,6306.250000,13843.250000,6524.000000
max,70.000000,70.000000,12690.000000,29526.000000,13971.000000,13421.000000,30544.000000,12727.000000,10010.000000,30103.000000,...,14182.000000,14456.000000,27249.000000,15946.000000,13645.000000,18969.000000,13790.000000,10197.000000,25687.000000,11670.000000


3) Coffee shop venues for each neighbourhood from Foursquare API. 

The venue search query will be used to find the number of coffee shops within each foot traffic sample point. The call will return a JSON file. The JSON file will return an eleement totalResults.

JSON data will be returned from the foursquare API called and the number of coffee venues will be added as as column to the df_ped dataframe

Initalising foursquare:

In [13]:
CLIENT_ID = 'DMQJLIDJ0EYXZOY2VVA52MB1A5HHX03WA5S0YF54QQDIVCW5' # your Foursquare ID
CLIENT_SECRET = 'MZTDY3U4BC2AGDMNAFBSOH34SHPVYDVKNYND32WBOETW3A1I' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

Example of foursquare call for the number of coffee shops 100m from a latlng value

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 50 # define radius we assume 50m is competitive business

CATID = '4bf58dd8d48988d1e0931735'

lat = '40.70463665187371'

lng = '-74.01155687409947'

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
lat, 
lng, 
radius, 
LIMIT,
CATID)
            
# make the GET request
results = requests.get(url).json()

#["response"]["totalResults"]

results

{'meta': {'code': 200, 'requestId': '5f4c1b4e3732647dc65ad1c8'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'coffee shop',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 40.705086652323715,
    'lng': -74.01096437816007},
   'sw': {'lat': 40.70418665142371, 'lng': -74.01214937003887}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53723e71498e7892c5155c07',
       'name': 'Gregorys Coffee',
       'location': {'address': '80 Broad St',
        'crossStreet': 'btwn Beaver & S William St',
        'l

Preparing the latlng value for GeoPy by removing words "Point" and brackets from each line in df_ped and replacing space with a comma

In [15]:
df_ped["the_geom"] = df_ped["the_geom"].str.extract('([-]\d+.\d+\s\d+.\d+)', expand=False)
df_ped["the_geom"] = df_ped["the_geom"].str.replace(' ', ',')
df_ped.head()

,Borough,the_geom,OBJECTID,Loc,Street_Nam,From_Stree,To_Street,Index,May07_AM,May07_PM,...,Sept17_PM,Sept17_MD,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,May19_AM,May19_PM,May19_MD
34,Manhattan,"-74.01155687409947,40.70463665187371",35,35,Broad Street,Beaver Street,South William Street,Y,3469,3992,...,"8,303","2,036","4,374","6,603","1,756","6,471","2,010","4,100","7,302","1,669"
35,Manhattan,"-74.01286204592034,40.70634164448266",36,36,Broadway,Morris Street,Exchange Place,Y,3660,8390,...,"12,650","8,126","5,221","19,725","6,818","10,726","9,615","5,049","11,765","7,029"
36,Manhattan,"-73.98219706247882,40.77181340301184",37,37,Broadway,West 63rd Street,West 64th Street,Y,1611,6764,...,"9,305","4,663","2,059","6,194","6,037","7,773","5,259","1,696","6,864","4,907"
37,Manhattan,"-74.01009312926121,40.715904559004194",38,38,Chambers Street,West Broadway,Greenwich Street,Y,7081,8512,...,"9,937","3,302","8,323","8,960","3,630","10,456","3,493","3,075","6,598","2,934"
38,Manhattan,"-73.97713579908014,40.7796808276313",39,39,Columbus Avenue,West 75th Street,West 76th Street,N,1071,3037,...,"3,626","2,977","1,524","3,905","3,780","3,794","2,451",-,-,-


Changing data type to string, removing commas and then converting to int. 

Original problem in that original column data types a mix between string and object.

In [16]:
months = ['May17_AM', 'May17_MD', 'May17_PM', 'Sept17_AM', 'Sept17_MD', 'Sept17_PM', 'May18_AM', 'May18_PM', 'May18_MD', 'Sept18_PM', 'Sept18_MD']
df_ped[months].dtypes

May17_AM     object
May17_MD     object
May17_PM     object
Sept17_AM    object
Sept17_MD    object
Sept17_PM    object
May18_AM     object
May18_PM     object
May18_MD     object
Sept18_PM    object
Sept18_MD    object
dtype: object

Making the data type change described above

In [17]:
df_ped[months] = df_ped[months].astype("str")

df_ped['May17_AM'] = df_ped['May17_AM'].str.replace(',','')
df_ped['May17_MD'] = df_ped['May17_MD'].str.replace(',','')
df_ped['May17_PM'] = df_ped['May17_PM'].str.replace(',','')
df_ped['Sept17_AM'] = df_ped['Sept17_AM'].str.replace(',','')
df_ped['Sept17_MD'] = df_ped['Sept17_MD'].str.replace(',','')
df_ped['Sept17_PM'] = df_ped['Sept17_PM'].str.replace(',','')
df_ped['May18_AM'] = df_ped['May18_AM'].str.replace(',','')
df_ped['May18_PM'] = df_ped['May18_PM'].str.replace(',','')
df_ped['May18_MD'] = df_ped['May18_MD'].str.replace(',','')
df_ped['Sept18_PM'] = df_ped['Sept18_PM'].str.replace(',','')
df_ped['Sept18_MD'] = df_ped['Sept18_MD'].str.replace(',','')

df_ped[months] = df_ped[months].astype("int32")
df_ped[months].dtypes

May17_AM     int32
May17_MD     int32
May17_PM     int32
Sept17_AM    int32
Sept17_MD    int32
Sept17_PM    int32
May18_AM     int32
May18_PM     int32
May18_MD     int32
Sept18_PM    int32
Sept18_MD    int32
dtype: object

Adding Neighbourhood Column to df_ped

In [18]:
df_ped["Neighbourhood"] = ""
df_ped.head()

,Borough,the_geom,OBJECTID,Loc,Street_Nam,From_Stree,To_Street,Index,May07_AM,May07_PM,...,Sept17_MD,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,May19_AM,May19_PM,May19_MD,Neighbourhood
34,Manhattan,"-74.01155687409947,40.70463665187371",35,35,Broad Street,Beaver Street,South William Street,Y,3469,3992,...,2036,4374,6603,1756,6471,2010,"4,100","7,302","1,669",
35,Manhattan,"-74.01286204592034,40.70634164448266",36,36,Broadway,Morris Street,Exchange Place,Y,3660,8390,...,8126,5221,19725,6818,10726,9615,"5,049","11,765","7,029",
36,Manhattan,"-73.98219706247882,40.77181340301184",37,37,Broadway,West 63rd Street,West 64th Street,Y,1611,6764,...,4663,2059,6194,6037,7773,5259,"1,696","6,864","4,907",
37,Manhattan,"-74.01009312926121,40.715904559004194",38,38,Chambers Street,West Broadway,Greenwich Street,Y,7081,8512,...,3302,8323,8960,3630,10456,3493,"3,075","6,598","2,934",
38,Manhattan,"-73.97713579908014,40.7796808276313",39,39,Columbus Avenue,West 75th Street,West 76th Street,N,1071,3037,...,2977,1524,3905,3780,3794,2451,-,-,-,


Function for getting the neighbourhood of a lat/lng value

Function splits the lnglat string, reverses it into a latlng, finds an address based on the latlng, splits this into a list and finds the first intersecting value in the with df_manhattan, because we don't know exactly what line the "Neighbourhood" appears on the address.

In [19]:
def getNeighbourhood(loc):
    geolocator = Nominatim(user_agent="New York")
    lnglat = loc.split(',')
    latlng = lnglat[::-1]
    address = geolocator.reverse(latlng)
    neighbourhood_names = df_manhattan["Name of the neighborhood"].values.tolist()
    neighbourhood = [value for value in neighbourhood_names if value in list(map(str.strip, address.address.split(',')))] 
    
    if len(neighbourhood) > 0:
        return neighbourhood[0]
        print("latlng = {}, found neighbourhood = {}".format(latlng, neighbourhood[0]))
    else:
        return ""
        print("latlng = {}, found neighbourhood = {}".format(latlng, "matching neighbourhood not found"))

Populating the Neighbourhood column on df_ped.

In [20]:
df_ped['Neighbourhood'] = df_ped['the_geom'].apply(getNeighbourhood)

Neighbourhood column of df_ped successfully altered, with the exception of latlng ['40.75284360634033', '-73.98335689692979']. This is a longitude for the Neighbourhood "South Midtown" which is not included in the df_manhattan list of manhattan neighbourhoods. This line will have the Neighbourhood value "Midtown South" manually assigned to it. There is possibly an error in the original list of wikipedia neighbourhoods seeing that Midtown is listed twice. One of these may be Midtown South. 

In [21]:
df_ped.at[57, 'Neighbourhood'] = 'Midtown South'

Checking change has been made

In [22]:
df_ped['Neighbourhood'].loc[57]

'Midtown South'

Function to reverse a longitude latitude value to be used for plotting points on map

In [23]:
def reverse(latlng):
    loc = latlng.split(',')
    lng = loc[1]
    lat = loc[0]
    lnglat = lng + "," + lat
    return lnglat

In [24]:
df_ped["latlng"] = df_ped["the_geom"].apply(reverse)
df_ped.head()

,Borough,the_geom,OBJECTID,Loc,Street_Nam,From_Stree,To_Street,Index,May07_AM,May07_PM,...,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,May19_AM,May19_PM,May19_MD,Neighbourhood,latlng
34,Manhattan,"-74.01155687409947,40.70463665187371",35,35,Broad Street,Beaver Street,South William Street,Y,3469,3992,...,4374,6603,1756,6471,2010,"4,100","7,302","1,669",Financial District,"40.70463665187371,-74.01155687409947"
35,Manhattan,"-74.01286204592034,40.70634164448266",36,36,Broadway,Morris Street,Exchange Place,Y,3660,8390,...,5221,19725,6818,10726,9615,"5,049","11,765","7,029",Financial District,"40.70634164448266,-74.01286204592034"
36,Manhattan,"-73.98219706247882,40.77181340301184",37,37,Broadway,West 63rd Street,West 64th Street,Y,1611,6764,...,2059,6194,6037,7773,5259,"1,696","6,864","4,907",Lincoln Square,"40.77181340301184,-73.98219706247882"
37,Manhattan,"-74.01009312926121,40.715904559004194",38,38,Chambers Street,West Broadway,Greenwich Street,Y,7081,8512,...,8323,8960,3630,10456,3493,"3,075","6,598","2,934",Tribeca,"40.715904559004194,-74.01009312926121"
38,Manhattan,"-73.97713579908014,40.7796808276313",39,39,Columbus Avenue,West 75th Street,West 76th Street,N,1071,3037,...,1524,3905,3780,3794,2451,-,-,-,Upper West Side,"40.7796808276313,-73.97713579908014"


Finding mean value for foot trafffic over 2017 to 2018 as this is a complete dataset for all df_pd locations

In [25]:
columns = ['Neighbourhood', 'latlng', 'Street_Nam', 'May17_AM', 'May17_MD', 'May17_PM', 'Sept17_AM', 'Sept17_MD', 'Sept17_PM', 'May18_AM', 'May18_PM', 'May18_MD', 'Sept18_PM', 'Sept18_MD']
df_ped_ra = df_ped[columns]
df_ped_ra['Mean'] = round(df_ped_ra.mean(axis=1),0)
df_ped_ra.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Neighbourhood,latlng,Street_Nam,May17_AM,May17_MD,May17_PM,Sept17_AM,Sept17_MD,Sept17_PM,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,Mean
34,Financial District,"40.70463665187371,-74.01155687409947",Broad Street,3522,1715,6747,4924,2036,8303,4374,6603,1756,6471,2010,4406.0
35,Financial District,"40.70634164448266,-74.01286204592034",Broadway,6274,9265,14040,7798,8126,12650,5221,19725,6818,10726,9615,10023.0
36,Lincoln Square,"40.77181340301184,-73.98219706247882",Broadway,1962,5446,6832,2091,4663,9305,2059,6194,6037,7773,5259,5238.0
37,Tribeca,"40.715904559004194,-74.01009312926121",Chambers Street,6923,2994,9546,8401,3302,9937,8323,8960,3630,10456,3493,6906.0
38,Upper West Side,"40.7796808276313,-73.97713579908014",Columbus Avenue,1558,2690,3627,1592,2977,3626,1524,3905,3780,3794,2451,2866.0


Mean for each neighbourhood - there are well over 50 coffee shops for each neighbourhood which is the limit for most foursquare api search calls, so we're going to plot foot traffic sample points instead

In [46]:
final_columns = ['Neighbourhood', 'latlng', 'Street_Nam', 'Mean']
df_ped_manhattan = df_ped_ra[final_columns]
df_ped_manhattan.head()

,Neighbourhood,latlng,Street_Nam,Mean
34,Financial District,"40.70463665187371,-74.01155687409947",Broad Street,4406.0
35,Financial District,"40.70634164448266,-74.01286204592034",Broadway,10023.0
36,Lincoln Square,"40.77181340301184,-73.98219706247882",Broadway,5238.0
37,Tribeca,"40.715904559004194,-74.01009312926121",Chambers Street,6906.0
38,Upper West Side,"40.7796808276313,-73.97713579908014",Columbus Avenue,2866.0


Finding Latitude and Longitude of Manhattan using GeoPy

In [26]:
address = 'Manhattan'

geolocator = Nominatim(user_agent = "New York")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


Creating a map of Manhattan with each pedestrian sample point superimposed on top. PROBLEM IN ONLY ONE VALUE, HUDSON RIVER GREENWAY is actually being shown on the map.

In [27]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for latlng, Mean, Street_Nam, Neighbourhood in zip(df_ped_ra['latlng'], df_ped_ra['Mean'], df_ped_ra['Street_Nam'],df_ped_ra['Neighbourhood']):
    label = '{}, {}'.format(Street_Nam, Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        list(map(float, latlng.split(","))),
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Finding the number of coffee shops for each foot traffic sample point in Manhattan, with a maximum of 100

Defining Foursquare parameters

In [28]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 100 # define radius we assume 50m is competitive business

CATID = '4bf58dd8d48988d1e0931735'

Defining a function for counting the number of coffee shops for each foot traffic sample point

In [29]:
def GetCoffeeVenueCount(latlng):
    lat = latlng.split(',')[0]
    lng = latlng.split(',')[1]
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
lat, 
lng, 
radius, 
LIMIT,
CATID)
    
    print('latitude:{} longitude:{}'.format(lat,lng))
    
    results = requests.get(url).json()["response"]["totalResults"]

    return results

Applying function

In [31]:
df_ped_ra["Coffee Venue Count"] = df_ped_ra["latlng"].apply(GetCoffeeVenueCount)
df_ped_ra.head()

latitude:40.70463665187371 longitude:-74.01155687409947
latitude:40.70634164448266 longitude:-74.01286204592034
latitude:40.77181340301184 longitude:-73.98219706247882
latitude:40.715904559004194 longitude:-74.01009312926121
latitude:40.7796808276313 longitude:-73.97713579908014
latitude:40.71898935935355 longitude:-73.98936305063111
latitude:40.735599264238246 longitude:-73.99265748471083
latitude:40.751694735123834 longitude:-73.97665547944715
latitude:40.731436017707026 longitude:-73.99441204702927
latitude:40.75884828352126 longitude:-73.98861503057542
latitude:40.76135361091173 longitude:-73.97514418621735
latitude:40.71493552371373 longitude:-73.99451531515767
latitude:40.7105537674607 longitude:-74.0084036486757
latitude:40.7080627209577 longitude:-74.00451806937401
latitude:40.76126460795223 longitude:-73.96888735867235
latitude:40.771440622634515 longitude:-73.96568512678532
latitude:40.749996387066524 longitude:-73.99134284997088
latitude:40.70835161274032 longitude:-74.01257

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Neighbourhood,latlng,Street_Nam,May17_AM,May17_MD,May17_PM,Sept17_AM,Sept17_MD,Sept17_PM,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,Mean,Coffee Venue Count
34,Financial District,"40.70463665187371,-74.01155687409947",Broad Street,3522,1715,6747,4924,2036,8303,4374,6603,1756,6471,2010,4406.0,7
35,Financial District,"40.70634164448266,-74.01286204592034",Broadway,6274,9265,14040,7798,8126,12650,5221,19725,6818,10726,9615,10023.0,5
36,Lincoln Square,"40.77181340301184,-73.98219706247882",Broadway,1962,5446,6832,2091,4663,9305,2059,6194,6037,7773,5259,5238.0,4
37,Tribeca,"40.715904559004194,-74.01009312926121",Chambers Street,6923,2994,9546,8401,3302,9937,8323,8960,3630,10456,3493,6906.0,3
38,Upper West Side,"40.7796808276313,-73.97713579908014",Columbus Avenue,1558,2690,3627,1592,2977,3626,1524,3905,3780,3794,2451,2866.0,1


Min max normalising Coffee Venue Count and Mean foot traffic columns

In [32]:
df_ped_ra["CVC Normalised"] = (df_ped_ra["Coffee Venue Count"] - df_ped_ra["Coffee Venue Count"].min())/(df_ped_ra["Coffee Venue Count"].max()-df_ped_ra["Coffee Venue Count"].min())

df_ped_ra["Mean Normalised"] = (df_ped_ra["Mean"] - df_ped_ra["Mean"].min())/(df_ped_ra["Mean"].max()-df_ped_ra["Mean"].min())

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Calculating Score

In [43]:
df_ped_ra["Score"] = df_ped_ra["Mean Normalised"] - df_ped_ra["CVC Normalised"]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Normalising Score

In [44]:
df_ped_ra["Score Normalised"] = round((df_ped_ra["Score"] - df_ped_ra["Score"].min()) / (df_ped_ra["Score"].max() - df_ped_ra["Score"].min())*10,0)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Colour coding for Score to be used in Map

In [45]:
df_ped_ra['marker_color'] = pd.cut(df_ped_ra['Score Normalised'], bins=10, labels=['darkred', 'red', 'orange', 'lightred', 'darkblue', 'blue', 'lightblue', 'darkgreen',  'green', 'lightgreen'])

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for latlng, Score, Street_Nam, Neighbourhood, marker_color in zip(df_ped_ra['latlng'], df_ped_ra['Score Normalised'], df_ped_ra['Street_Nam'],df_ped_ra['Neighbourhood'], df_ped_ra["marker_color"]):
    label = '{}, {}, {}'.format(Score, Street_Nam, Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        list(map(float, latlng.split(","))),
        radius=5,
        popup=label,
        color= marker_color,
        fill=True,
        fill_color= marker_color,
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [47]:
df_table = df_ped_ra[['Neighbourhood', 'Street_Nam', 'Score Normalised', 'marker_color']]
df_table.sort_values(by=['Score Normalised'], ascending = False, inplace= True)
df_table.reset_index(drop = True, inplace=True)
df_table

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Neighbourhood,Street_Nam,Score Normalised,marker_color
0,Theater District,Eighth Avenue,10.0,lightgreen
1,Midtown,Fifth Avenue,10.0,lightgreen
2,Garment District,West 34th Street,10.0,lightgreen
3,Harlem,West 125th Street,9.0,green
4,Midtown East,Park Avenue,7.0,lightblue
5,Midtown South,West 40th Street,7.0,lightblue
6,Washington Heights,West 181st Street,7.0,lightblue
7,Hell's Kitchen,Hudson River Greenway,7.0,lightblue
8,Upper West Side,Columbus Avenue,6.0,blue
9,Midtown East,Madison Avenue,6.0,blue


In [48]:
df_ped_ra

,Neighbourhood,latlng,Street_Nam,May17_AM,May17_MD,May17_PM,Sept17_AM,Sept17_MD,Sept17_PM,May18_AM,...,May18_MD,Sept18_PM,Sept18_MD,Mean,Coffee Venue Count,CVC Normalised,Mean Normalised,Score,Score Normalised,marker_color
34,Financial District,"40.70463665187371,-74.01155687409947",Broad Street,3522,1715,6747,4924,2036,8303,4374,...,1756,6471,2010,4406.0,7,1.000000,0.211053,-0.788947,1.0,darkred
35,Financial District,"40.70634164448266,-74.01286204592034",Broadway,6274,9265,14040,7798,8126,12650,5221,...,6818,10726,9615,10023.0,5,0.714286,0.518093,-0.196192,5.0,darkblue
36,Lincoln Square,"40.77181340301184,-73.98219706247882",Broadway,1962,5446,6832,2091,4663,9305,2059,...,6037,7773,5259,5238.0,4,0.571429,0.256532,-0.314896,4.0,lightred
37,Tribeca,"40.715904559004194,-74.01009312926121",Chambers Street,6923,2994,9546,8401,3302,9937,8323,...,3630,10456,3493,6906.0,3,0.428571,0.347710,-0.080862,6.0,blue
38,Upper West Side,"40.7796808276313,-73.97713579908014",Columbus Avenue,1558,2690,3627,1592,2977,3626,1524,...,3780,3794,2451,2866.0,1,0.142857,0.126872,-0.015985,6.0,blue
39,Lower East Side,"40.71898935935355,-73.98936305063111",Delancey Street,1647,3295,4534,1634,2420,3238,1851,...,3249,4010,3034,3062.0,7,1.000000,0.137586,-0.862414,0.0,darkred
40,Union Square,"40.735599264238246,-73.99265748471083",East 14th Street,3303,10204,18021,4251,5078,19584,2601,...,6924,11932,6408,9312.0,4,0.571429,0.479228,-0.092200,6.0,blue
41,Midtown East,"40.751694735123834,-73.97665547944715",East 42nd Street,9241,6428,20258,8551,6279,16529,8452,...,5873,23025,8808,12131.0,5,0.714286,0.633322,-0.080963,6.0,blue
42,Greenwich Village,"40.731436017707026,-73.99441204702927",East 8th Street,1352,2682,4866,1212,1690,5016,905,...,2276,5929,3014,3092.0,2,0.285714,0.139226,-0.146488,5.0,darkblue
43,Theater District,"40.75884828352126,-73.98861503057542",Eighth Avenue,8389,9745,25824,7802,12536,23979,9734,...,11806,24112,9241,15589.0,3,0.428571,0.822346,0.393775,10.0,lightgreen
